In [8]:
import io
import re
import string
import tqdm
import os

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

In [11]:
os.chdir("/home/sakshmeno/Documents/GitHub/C-RNN-approach/Vectors")

In [45]:
with open("SECURE_DATA.txt") as secure_data_obj:
    codeLines = secure_data_obj.read()

comment_lines = []
codeLines = codeLines.replace("\t","").split("\n")
multi_line_flag = 0
for line_number in range(len(codeLines)):
    if multi_line_flag:
        if codeLines[line_number].__contains__("*/"):
            multi_line_flag = 0
        comment_lines.append(line_number)
    elif codeLines[line_number].__contains__("/*"):
        comment_lines.append(line_number)
        multi_line_flag = 1
    elif codeLines[line_number].startswith("//"):
        comment_lines.append(line_number)
    elif codeLines[line_number].__contains__("//"):
        comment_start = codeLines[line_number].find('//')
        codeLines[line_number] = codeLines[line_number][:comment_start]
        
comment_lines.reverse()
for i in comment_lines:
    codeLines.pop(i)

In [54]:
vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0  # add a padding token

for line in codeLines:
  tokens = list(line.lower().split())
  for token in tokens:
    if token not in vocab:
      vocab[token] = index
      index += 1
vocab_size = len(vocab)
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: '<pad>', 1: '#include', 2: '<compiler.h>', 3: '<timedelay.h>', 4: '"hardwareprofile.h"', 5: '"peripherals/nand.h"', 6: '"peripherals/accel.h"', 7: '#ifdef', 8: 'use_gyro', 9: '"peripherals/gyro.h"', 10: '#endif', 11: '"peripherals/rtc.h"', 12: '"usb/usb.h"', 13: '"usb/usb_function_msd.h"', 14: '"usb/usb_function_cdc.h"', 15: '"mdd', 16: 'file', 17: 'system/fsio.h"', 18: '"usb/usb_cdc_msd.h"', 19: '"ftl/fsftl.h"', 20: '"utils/fsutils.h"', 21: '"utils/fifo.h"', 22: '"utils/util.h"', 23: '"peripherals/analog.h"', 24: '"settings.h"', 25: '"logger.h"', 26: '"ax3', 27: 'selftest.h"', 28: 'void', 29: 'runattached(void);', 30: 'runlogging(void);', 31: 'timedtasks(void);', 32: 'ledtasks(void);', 33: 'runtestsequence(void);', 34: '__attribute__((interrupt,auto_psv))', 35: '_defaultinterrupt(void)', 36: '{', 37: 'static', 38: 'unsigned', 39: 'int', 40: 'intcon1val;', 41: 'led_set(led_magenta);', 42: 'intcon1val', 43: '=', 44: 'intcon1;', 45: 'nop();', 46: 'intcon1', 47: '0;', 48: 'reset();', 

In [55]:
len(inverse_vocab.keys())
# inverse_vocab.size

2147